In [23]:
# Import glob, a module that helps with file management.
import glob
import pandas as pd
from spellchecker import SpellChecker
from pathlib import Path
from nltk import word_tokenize

In [24]:
# Load the spellchecker dictionary.
# Replace the language attribute with another 2 letter code
# to select another language. Options are: English - ‘en’, Spanish - ‘es’,
# French - ‘fr’, Portuguese - ‘pt’, German - ‘de’, Russian - ‘ru’.

spell = SpellChecker(language='fr')

In [28]:
### Dictionary Test a Folder of .txt Files ###

# We'll use Pandas to create a dataframe (a table) that can hold 
# information about an OCR'ed page and display it in a tabular format.
# This dataframe will start out empty with only its column headers 
# defined. We'll add information to it one page at a time. So each
# row will represent 1 page.

placeholder_values = ['placeholder', 0, 0, 0, 'placeholder', 'placeholder']

df = pd.DataFrame([placeholder_values], columns=["file_name","token_count","unknown_count","readability","unknown_words","text"])

# Set the folder for the input images
texts_folder = Path('./output/')

for txt_file in texts_folder.rglob('*.txt'):
    
    # Open each text file and read text into `ocrText`
    with open(txt_file, 'r') as inputFile:
        ocrText = inputFile.read()
        
    # Join hyphenated words that are split between lines by 
    # looking for a hyphen followed by a newline character: "-\n"
    # "\n" is an "escape character" and represents the 
    # "newline," a character that is usually invisible 
    # to human readers but that computers use to mark the 
    # end/beginning of a line. Each time you press the 
    # Enter/Return key on your keyboard, an invisible "\n" 
    # is created to mark the beginning of a new line.
    ocrText = ocrText.replace("-\n","")
    
    # First, we'll use NLTK to "tokenize" text. 
            # "Tokenize" here means to take a page of our OCR'ed text,
            # which Python is currently reading as one big glob of data,
            # and separate each word out so that it can be read as an
            # individual piece of data within a larger data structure 
            # (a list). This process also removes punctuation.
    tokens = word_tokenize(ocrText)
    
    # Lowercase all tokens
    tokens = [token.lower() for token in tokens if token.isalpha()]
    
    # Now we can get all of the words that don't match the 
    # spellchecker dictionary or our list of place names--
    # these are the potential spelling errors.
    unknown = spell.unknown(tokens)
    
    # Let's use a little math to find out how many potential 
    # spelling errors were identified. As part of this process, 
    # we'll create a "readability" score that will give us a 
    # percentage of how readable each file is--how much of the 
    # OCR'ed is "correct."
        
    # If the list of unknown tokens (words) is greater than 0 
    # (i.e. if the list is not empty):
    if len(unknown) != 0:
            
               # Following order of operations, here's what's happening 
               # in the readability variable below:
               # 1. Divide the number of unknown tokens (len(unknown)) 
                    # by the total number of tokens on the page
                    # (len(tokens)). Use "float" to specify that Python
                    # returns a decimal number:
                        # (float(len(unknown))/float(len(tokens))
               # 2. Multiply the number from step 1 by 100.
                    # (float(len(unknown))/float(len(tokens)) * 100)
               # 3. Subtract the number from step 2 from 100.
                    # 100 - (float(len(unknown))/float(len(tokens)) * 100)
               # 4. Round the number from step 3 to 2 decimal places
                    # round(100 - (float(len(unknown))/float(len(tokens)) * 100), 2)
            
        readability = round(100 - (float(len(unknown))/float(len(tokens)) * 100), 2)
        
        # If the list of unknown tokens is empty (or equal to 0), then readability is 100!
    else:
        readability = 100
    
    # Let's create a record of the readability information 
    # for this page that we'll add to the dataframe. 
    # The following is a Python dictionary, another way of 
    # storing data. Each word or phrase to the left of the : is a
    # "key" -- think of it as a column header. Each piece of 
    # information to the right is a "value" -- information 
    # written in a single cell below each header. 

    df2 = pd.DataFrame({
            "file_name" : txt_file.as_posix(),
            "token_count" : len(tokens),
            "unknown_count" : len(unknown),
            "readability" : readability,
            "unknown_words" : [unknown],
            "text" : ocrText
            })

    df = pd.concat([df, df2])

    # This statement lets us know if a page has been successfully 
    # checked for readability.
    print(txt_file, "checked for readability.")
    
# This time, instead of creating individual .txt files for each page,
# we're going to save all of the OCR'ed text and readability 
# information to a single .csv ("comma separated value") file. 
# We can view this file format as a table. Having everything stored 
# like this will help us with clean up and future analysis.
df.to_csv(f'{texts_folder}/spellcheck_data.csv', header=True, index=False, sep=',')

# We have the data stored in a file now, but we can also 
# preview it here:
df

output/Démonomanie I.1.txt checked for readability.
output/.ipynb_checkpoints/Démonomanie I.1-checkpoint.txt checked for readability.


,file_name,token_count,unknown_count,readability,unknown_words,text
0,placeholder,0,0,0.00,placeholder,placeholder
0,output/Démonomanie I.1.txt,2518,903,64.14,"{ip, parleau, oïes, ption, mefprife, helas, en...",\n\n à p cS LV - : { - 2 e. / * 4 \nz ...
0,output/.ipynb_checkpoints/Démonomanie I.1-chec...,2518,903,64.14,"{ip, parleau, oïes, ption, mefprife, helas, en...",\n\n à p cS LV - : { - 2 e. / * 4 \nz ...


In [ ]:
# Delete the df variable in case we wish to run this script again
del df

In [ ]:
# Identify the sample_output file path.
# Remember that our readability output is also stored 
# in this file as a .csv. We don't want to change it, 
# so we'll use glob to look for only .txt files.
filePath = glob.glob(f"{outDir}/*.txt")

outputfile_path = Path("./final")

outputfile_path.mkdir(exist_ok=True)

outputfile = f'{outputfile_path.as_posix()}/{doc_name}_corrected.txt'

# Apply the following loop to one file at a time in filePath.
for file in filePath:
    
    # Open a file in "read" (r) mode.
    text = open(file, "r")
    
    # Read in the contents of that file.
    text = text.read()
    
    # Find instances of and unknown word and replace
    # with a known word.
    
    unknown_word = "|"
    
    known_word = ""
  
    word_correction = text.replace(unknown_word, known_word)

    print("All instances of " + unknown_word + " replaced with " + known_word + ".")

    unknown_word = "EE"
    
    known_word = ""
  
    word_correction = word_correction.replace(unknown_word, known_word)

    print("All instances of " + unknown_word + " replaced with " + known_word + ".")
    
    # Reopen the file in "write" (w) mode.
    file = open(outputfile, "w")
    
    # Add the changed word into the reopened file.
    file.write(word_correction)
    
    # Close the file.
    file.close()

In [ ]:
### Dictionary Test a Folder of .txt Files ###

# We'll use Pandas to create a dataframe (a table) that can hold 
# information about an OCR'ed page and display it in a tabular format.
# This dataframe will start out empty with only its column headers 
# defined. We'll add information to it one page at a time. So each
# row will represent 1 page.

placeholder_values = ['placeholder', 0, 0, 0, 'placeholder', 'placeholder']

df = pd.DataFrame([placeholder_values], columns=["file_name","token_count","unknown_count","readability","unknown_words","text"])

# Set the folder for the input images
texts_folder = Path("./final")

for txt_file in texts_folder.rglob('*.txt'):
    
    # Open each text file and read text into `ocrText`
    with open(txt_file, 'r') as inputFile:
        ocrText = inputFile.read()
        
    # Join hyphenated words that are split between lines by 
    # looking for a hyphen followed by a newline character: "-\n"
    # "\n" is an "escape character" and represents the 
    # "newline," a character that is usually invisible 
    # to human readers but that computers use to mark the 
    # end/beginning of a line. Each time you press the 
    # Enter/Return key on your keyboard, an invisible "\n" 
    # is created to mark the beginning of a new line.
    ocrText = ocrText.replace("-\n","")
    
    # First, we'll use NLTK to "tokenize" text. 
            # "Tokenize" here means to take a page of our OCR'ed text,
            # which Python is currently reading as one big glob of data,
            # and separate each word out so that it can be read as an
            # individual piece of data within a larger data structure 
            # (a list). This process also removes punctuation.
    tokens = word_tokenize(ocrText)
    
    # Lowercase all tokens
    tokens = [token.lower() for token in tokens if token.isalpha()]
    
    # Now we can get all of the words that don't match the 
    # spellchecker dictionary or our list of place names--
    # these are the potential spelling errors.
    unknown = spell.unknown(tokens)
    
    # Let's use a little math to find out how many potential 
    # spelling errors were identified. As part of this process, 
    # we'll create a "readability" score that will give us a 
    # percentage of how readable each file is--how much of the 
    # OCR'ed is "correct."
        
    # If the list of unknown tokens (words) is greater than 0 
    # (i.e. if the list is not empty):
    if len(unknown) != 0:
            
               # Following order of operations, here's what's happening 
               # in the readability variable below:
               # 1. Divide the number of unknown tokens (len(unknown)) 
                    # by the total number of tokens on the page
                    # (len(tokens)). Use "float" to specify that Python
                    # returns a decimal number:
                        # (float(len(unknown))/float(len(tokens))
               # 2. Multiply the number from step 1 by 100.
                    # (float(len(unknown))/float(len(tokens)) * 100)
               # 3. Subtract the number from step 2 from 100.
                    # 100 - (float(len(unknown))/float(len(tokens)) * 100)
               # 4. Round the number from step 3 to 2 decimal places
                    # round(100 - (float(len(unknown))/float(len(tokens)) * 100), 2)
            
        readability = round(100 - (float(len(unknown))/float(len(tokens)) * 100), 2)
        
        # If the list of unknown tokens is empty (or equal to 0), then readability is 100!
    else:
        readability = 100
    
    # Let's create a record of the readability information 
    # for this page that we'll add to the dataframe. 
    # The following is a Python dictionary, another way of 
    # storing data. Each word or phrase to the left of the : is a
    # "key" -- think of it as a column header. Each piece of 
    # information to the right is a "value" -- information 
    # written in a single cell below each header. 

    df2 = pd.DataFrame({
            "file_name" : txt_file.as_posix(),
            "token_count" : len(tokens),
            "unknown_count" : len(unknown),
            "readability" : readability,
            "unknown_words" : [unknown],
            "text" : ocrText
            })

    df = pd.concat([df, df2])

    # This statement lets us know if a page has been successfully 
    # checked for readability.
    print(txt_file, "checked for readability.")
    
# This time, instead of creating individual .txt files for each page,
# we're going to save all of the OCR'ed text and readability 
# information to a single .csv ("comma separated value") file. 
# We can view this file format as a table. Having everything stored 
# like this will help us with clean up and future analysis.
df.to_csv(f'{texts_folder}/spellcheck_data2.csv', header=True, index=False, sep=',')

# We have the data stored in a file now, but we can also 
# preview it here:
df

In [ ]:
# Delete the df variable in case we wish to run this script again
del df